# You Work at the North Pole
Every day, little children around the world write letters to Santa telling him what they want for christmas. Santa reads these letters, and if the child is nice, the gift gets made by one of the elves, wrapped, and put in santa's sleigh to be delivered on Christmas Eve. If the child is naughty, Santa puts coal in their stocking. Coal doesn't need to be wrapped, but it does need to be put in Santa's sleigh.

In an attempt to teach myself threading in python, I'm going to create a working, multi-threaded model of Santa's workshop. 

## All By My Elf
For now, let's assume Santa's workshop has undergone some budget cuts, and only has one elf employed. Making a present takes a variable amount of time--we'll just pass the amount of time with the wish. Wrapping a present takes a constant amount of time: 10 minutes. For now, we're just going to assume all children are good, so these two tasks can be represented by the following functions:

In [1]:
import time
import random

def make_wish_come_true(wish, construction_time):
    time.sleep(construction_time)
    return wish
        
def wrap_present(present):
    time.sleep(0.1)
    return present

So to fill Santa's sleigh, our lone elf gathers the letters, makes the toys, wraps the toys, and puts them in the sleigh. If a child is naughty, we'll just toss some coal directly into the sleigh; no use wasting an elf's precious time on a bratty kid.

In [2]:
import queue

def fill_santas_sleigh(childrens_letters, santas_list):
    # Santa gives nice kids' letters to elves so they can make them into toys.
    letters = []
    # Finished toys sit in a pile waiting to be wrapped.
    toys = []
    # Finished, wrapped toys.
    sleigh = []
    
    for child, wish in childrens_letters.items():
        # He's makin' a list...checkin' it twice!
        if santas_list[child] == "Naughty" and santas_list[child] == "Naughty":
            sleigh.append("Coal for " + child)
        else:
            letters.append(wish + " for " + child)
            
    while letters:
        toy = make_wish_come_true(letters.pop(), random.random()) # Random time between 0 and 1 hour
        toys.append(toy)

    while toys:
        present = wrap_present(toys.pop())
        sleigh.append(present)
    
    return sleigh

To make sure this all works the way we expect it to, here's some sample data and a test to run it through!

In [3]:
childrens_letters_2017 = {
    "Jimmy":"Racecar",
    "Alison":"Nerf gun",
    "Steve":"Bicycle",
    "Gwendylyn":"Skateboard",
    "Lewis":"Action figure",
    "Kate":"Nintendo",
    "Alex":"Coloring book",
    "Sue":"Stuffed animal"
}

santas_list_2017 = {
    "Jimmy":"Nice",
    "Alison":"Nice",
    "Steve":"Naughty",
    "Gwendylyn":"Nice",
    "Lewis":"Naughty",
    "Kate":"Naughty",
    "Alex":"Nice",
    "Sue":"Nice"
}

sleigh_2017 = fill_santas_sleigh(childrens_letters_2017, santas_list_2017)

print (sleigh_2017)

['Coal for Steve', 'Coal for Lewis', 'Coal for Kate', 'Racecar for Jimmy', 'Nerf gun for Alison', 'Skateboard for Gwendylyn', 'Coloring book for Alex', 'Stuffed animal for Sue']


## Thread The Halls
Ok, so the workshop is back in the black, and we now have a bunch of elves, not just one. But our `fill_santas_sleigh` function still operates the same, and the new recruits are getting bored. Wouldn't it be great if multiple elves could make and wrap presents in parallel? 

This is where threading comes in. Each elf is a thread, and it can either make or wrap a gift. So if we have "n" elves, we would want each elf to operate like this:

1. If there are toys lying around waiting to be wrapped, wrap them and put them in the sleigh.
2. If there aren't any toys made, go get a letter from santa and make that wish come true.
3. If everything's made, wrapped, and in the sleigh, *exit* the workshop.

There's an important caveat to realize here: if there are multiple elves looking for something to do, they need to do different things. For example, we don't want our little workers bickering over letters or breaking toys because they're both trying to wrap the same one. The solution? Locks. 

We want to make sure only one elf talks to Santa at a time, so if Santa is already busy giving a letter to another elf, other elves should quietly wait their turn. We'll call this a `letter_lock`. Similarly, to avoid fights with elves wanting to wrap the same toy (or elves getting hit by toys being thrown onto the pile), only one elf should approach the pile of toys at a time. The other elves *politely waiting their turn*--meaning we'll also have a `toy_lock`. Finally, we don't want elves throwing toys into the sleigh at the same time--they could collide and break! Christmas would be ruined! To avoid this catastrophe, we'll have a `sleigh_lock`.

> ### A quick note on locks...
> Locks are really important if the resource you're trying to access isn't threadsafe. Lists are a good example of this, but it's worth noting that there are other classes (queues come to mind) that are indeed threadsafe, so you don't have to bother with locks. Therefore, I'm intentionally choosing a sub-optimal data structure to more completely illustrate how multithreading works.

\*cough cough\* With that non-metaphor-fitting blabber out of the way...let's create our own Elf class that is a subclass of Thread (found in the threading python module). During each elf's orientation training (`__init__` for all you programmers), he'll be given a badge with his name, get shown where to find Santa (who has the children's letters), the pile of toys, and Santa's sleigh. He'll also be told about the one-elf-at-a-time policies for each, and finally, given a deadline for how fast he has to complete toys.

In [4]:
import threading

class Elf (threading.Thread):
    def __init__(self, name, letter_location, toy_location, sleigh_location, letter_policy, toy_policy, sleigh_policy, toy_deadline):
        threading.Thread.__init__(self)
        self.name = name
        self.letters = letter_location
        self.toys = toy_location
        self.sleigh = sleigh_location
        self.turn_for_letter = letter_policy
        self.turn_for_toy = toy_policy
        self.turn_for_sleigh = sleigh_policy
        self.deadline = toy_deadline
        print (self.name, "is ready to make some toys!")
    
    def run(self):
        work_time = random.Random()
        while self.toys or self.letters:
            # Grab a toy and let the next elf take their turn.
            toy = None
            with self.turn_for_toy:
                if self.toys:
                    toy = self.toys.pop()
            # Wrap that toy and put it in the sleigh.
            if toy:
                present = wrap_present(toy)
                with self.turn_for_sleigh:
                    self.sleigh.append(present)
                print (self.name, "wrapped a", present, "and threw it in the sleigh")
                time.sleep(random.random())
                continue

            # Grab a letter and let the next elf take their turn.
            wish = None
            with self.turn_for_letter:
                if self.letters:
                    wish = self.letters.pop()
            # Make a toy and put it in the pile.
            if wish:
                toy = make_wish_come_true(wish, work_time.randrange(self.deadline))
                with self.turn_for_toy:
                    self.toys.append(toy)
                print (self.name, "made a", wish, "and put it in the toy pile")
                time.sleep(random.random())
                continue
                
        print (self.name, "is punching out for the day!")

You'll notice in the code above that our elf class has a `run` method that follows the 3 steps we outlined above for how an elf should behave. When the function finishes, the elf (thread) exits. Notice this also holds a lot of the code we'd previously placed in `fill_santas_sleigh`, which means we'll want to modify that code as well. The multi-threaded code looks like this:

In [5]:
santa_lock = threading.Lock()
toy_lock = threading.Lock()
sleigh_lock = threading.Lock()

# Maximum time it takes to make a toy
TOY_DEADLINE = 1.0 

def everyone_fill_santas_sleigh(childrens_letters, santas_list, elves_names):
    # Santa gives nice kids' letters to elves so they can make them into toys.
    letters = []
    # Finished toys sit in a pile waiting to be wrapped.
    toys = []
    # Finished, wrapped toys.
    sleigh = []
    
    for child, wish in childrens_letters.items():
        # He's makin' a list...checkin' it twice!
        if santas_list[child] == "Naughty" and santas_list[child] == "Naughty":
            sleigh.append("coal for " + child)
        else:
            letters.append(wish + " for " + child)
    
    elves = []
    for name in elves_names:
        elf = Elf(name, letters, toys, sleigh, santa_lock, toy_lock, sleigh_lock, TOY_DEADLINE)
        elf.start()
        elves.append(elf)

    # Wait for all the elves to finish working before closing the workshop for the day.
    for elf in elves:
        elf.join()
    
    time.sleep(0.1)
    print ("All clear! Shutting down the workshop for the day.")
    return sleigh

# It's A Christmas Miracle!
That's everything required for a multi-threaded version of the North Pole. To make sure everything's behaving as expected, here's our previous example, this time with the addition of 3 elves.

In [6]:
letters_2017 = {
    "Jimmy":"Racecar",
    "Alison":"Nerf gun",
    "Steve":"Bicycle",
    "Gwendylyn":"Skateboard",
    "Lewis":"Action figure",
    "Kate":"Nintendo",
    "Alex":"Coloring book",
    "Sue":"Stuffed animal"
}

santas_list_2017 = {
    "Jimmy":"Nice",
    "Alison":"Nice",
    "Steve":"Naughty",
    "Gwendylyn":"Nice",
    "Lewis":"Naughty",
    "Kate":"Naughty",
    "Alex":"Nice",
    "Sue":"Nice"
}

elves_names_2017 = {
    "Flint",
    "Peabody",
    "Keebler"
}

sleigh_2017 = everyone_fill_santas_sleigh(letters_2017, santas_list_2017, elves_names_2017)

print (sleigh_2017)

Peabody is ready to make some toys!
Keebler is ready to make some toys!
Peabody Flintmade a  is ready to make some toys!Stuffed animal for Sue
Flint  and put it in the toy pilemade a
 Coloring book for Alex and put it in the toy pile
Keebler wrapped a Stuffed animal for Sue and threw it in the sleigh
Flint wrapped a Coloring book for Alex and threw it in the sleigh
Peabody made a Skateboard for Gwendylyn and put it in the toy pile
Keebler made a Nerf gun for Alison and put it in the toy pile
Peabody wrapped a Skateboard for Gwendylyn and threw it in the sleigh
Keebler wrapped a Nerf gun for Alison and threw it in the sleigh
Flint made a Racecar for Jimmy and put it in the toy pile
Peabody wrapped a Racecar for Jimmy and threw it in the sleigh
Keebler is punching out for the day!
Flint Peabodyis punching out for the day! 
is punching out for the day!
All clear! Shutting down the workshop for the day.
['coal for Steve', 'coal for Lewis', 'coal for Kate', 'Stuffed animal for Sue', 'Colori

# ...And To All A Good Night
Just in case you wanna do some stress testing, here's a test that runs through a list of over 4500 nouns! (It's updated regularly, so it keeps growing)

In [7]:
from urllib.request import urlopen

# Some constants to help control the size of the test.
NUMBER_OF_ELVES = 100
MAX_NUMBER_OF_LETTERS = 4000

# This gets a big list of nouns and puts it in a list of wishes.
word_site = "http://www.desiquintans.com/downloads/nounlist/nounlist.txt"
response = urlopen(word_site)
noun_list = response.read().splitlines()
wishes = random.sample(noun_list, min(MAX_NUMBER_OF_LETTERS, len(noun_list)))

santas_elves = list(map(lambda num: "elf" + str(num), range(NUMBER_OF_ELVES)))
childrens_letters = {}
naughty_or_nice_list = {}

# Generate the children's letters and Santa's naughty/nice list.
for i, wish in enumerate(wishes):
    childs_name = "child" + str(i)
    child_is_nice = random.choice([True, True, True, False]) # Most children are nice
    childrens_letters[childs_name] = wish.decode('utf-8') # Decode gets rid of b' prefix
    naughty_or_nice_list[childs_name] = "Nice" if child_is_nice else "Naughty"

big_ass_sleigh = everyone_fill_santas_sleigh(childrens_letters, naughty_or_nice_list, santas_elves)
print (big_ass_sleigh)

elf0 is ready to make some toys!
elf1 is ready to make some toys!
elf2elf0  made a exile for child3999is ready to make some toys! 
and put it in the toy pile
elf3elf2  made a signet for child3998is ready to make some toys! 
and put it in the toy pileelf4
 is ready to make some toys!
elf5 is ready to make some toys!
elf4elf6  made ais ready to make some toys!
elf7  is ready to make some toys!
pinto for child3997elf6  elf8made aand put it in the toy pile  
is ready to make some toys!vegetable for child3996
 elf9and put it in the toy pile 
is ready to make some toys!elf8 made a 
headlight for child3994elf10  and put it in the toy pileis ready to make some toys!

elf11 is ready to make some toys!
elf10elf12 is ready to make some toys!
 made a elf13parachute for child3993elf12   is ready to make some toys!and put it in the toy pile

made aelf14  is ready to make some toys!
pipe for child3992 elf15and put it in the toy pile
 is ready to make some toys!
elf14 made a departure for child3991 an

elf64 made a technologist for child3928 and put it in the toy pile
elf10 wrapped a gumshoes for child3933 and threw it in the sleigh
elf46 made a opossum for child3927 and put it in the toy pile
elf13 made a direction for child3926 and put it in the toy pile
elf58 wrapped a concern for child3932 and threw it in the sleigh
elf45 wrapped a worker for child3930 and threw it in the sleigh
elf69 wrapped a ballpark for child3929 and threw it in the sleigh
elf16 made a manx for child3924 and put it in the toy pile
elf7 made a orange for child3921 and put it in the toy pile
elf87 made a heavy for child3919 and put it in the toy pile
elf51elf34  elf41wrapped awrapped a   made aopossum for child3927technologist for child3928   kitten for child3918and threw it in the sleighand threw it in the sleigh 

and put it in the toy pile
elf27 made a native for child3917 and put it in the toy pile
elf30 made aelf93  scarf for child3916wrapped a  and put it in the toy piledirection for child3926
 and threw 

elf45 wrapped a somewhere for child3835 and threw it in the sleigh
elf85 made a magazine for child3819 and put it in the toy pile
elf18 wrapped a education for child3833 and threw it in the sleigh
elf10 made a alibi for child3818 and put it in the toy pile
elf61 wrapped a blinker for child3832 and threw it in the sleigh
elf3 made a schedule for child3815 and put it in the toy pile
elf73 wrapped a ptarmigan for child3831 and threw it in the sleigh
elf97 made a increase for child3812 and put it in the toy pile
elf74 wrapped a kangaroo for child3830 and threw it in the sleigh
elf94 made a outlay for child3811 and put it in the toy pile
elf31 made a wafer for child3809 and put it in the toy pile
elf36elf1 wrapped a  wrapped aco-producer for child3822  heaven for child3825and threw it in the sleigh 
and threw it in the sleigh
elf69 wrapped a self for child3820 and threw it in the sleighelf87
 made a miter for child3807 and put it in the toy pile
elf16 made a boat for child3806 and put it in

elf74 made a elf30elf53walrus for child3717   wrapped amade aand put it in the toy pile  
satire for child3716 regulation for child3732and put it in the toy pile 
and threw it in the sleigh
elf85 elf77wrapped a  wrapped aaftermath for child3731  model for child3730and threw it in the sleigh 
and threw it in the sleigh
elf23 elf13 made awrapped a  acupuncture for child3715guitar for child3729  and put it in the toy pileand threw it in the sleigh

elf6 wrapped a crush for child3728 and threw it in the sleigh
elf61 wrapped a marsh for child3726 and threw it in the sleigh
elf63 wrapped a sign for child3725 and threw it in the sleigh
elf18 wrapped a center for child3723 and threw it in the sleigh
elf16 made a recommendation for child3714 and put it in the toy pile
elf79 made a hammock for child3713 and put it in the toy pile
elf26 made a difficulty for child3712 and put it in the toy pile
elf73 wrapped a shoestring for child3722 and threw it in the sleigh
elf67 wrapped a initiative for chil

elf96 made a luncheonette for child3615 elf43and put it in the toy pile 
wrapped a n for child3631 and threw it in the sleigh
elf75 wrapped a green for child3629 and threw it in the sleigh
elf71 made a pumpkin for child3614 and put it in the toy pile
elf80 wrapped a ground for child3628 and threw it in the sleigh
elf29 wrapped a remove for child3627 and threw it in the sleigh
elf9 made a master for child3613 and put it in the toy pile
elf99elf53  wrapped amade a  mercury for child3626surround for child3612  and threw it in the sleighand put it in the toy pileelf44

 made a jackal for child3611 and put it in the toy pile
elf85 wrapped a derrick for child3624 and threw it in the sleigh
elf20 wrapped a microwave for child3622 and threw it in the sleigh
elf45 made a racist for child3610 and put it in the toy pile
elf93 made a promotion for child3609 and put it in the toy pile
elf91 elf57wrapped a  made acribbage for child3620  constellation for child3608and threw it in the sleigh 
and put 

...

elf47 elf63elf33wrapped a   wrapped amade abook for child421   flesh for child419crowd for child409and threw it in the sleigh  and threw it in the sleighand put it in the toy pile


elf7 made a penalty for child403 and put it in the toy pile
elf53 wrapped a factor for child418 and threw it in the sleigh
elf80 made a rush for child396 and put it in the toy pile
elf27 wrapped a hygienic for child417 and threw it in the sleigh
elf48 made a type for child395 and put it in the toy pile
elf44 made a intention for child394 and put it in the toy pile
elf65 wrapped a petal for child415 and threw it in the sleigh
elf61 made a emu for child393 and put it in the toy pile
elf97 wrapped aelf10 pantyhose for child414  wrapped aand threw it in the sleigh 
scheme for child413 and threw it in the sleigh
elf56elf69  made a stockings for child392 wrapped aand put it in the toy pile birthday for child411 and threw it in the sleigh

elf5 made a outback for child390 and put it in the toy pile
elf34 wrapped a

elf10elf68 wrapped a  priority for child321made a  and threw it in the sleighfingernail for child309
 and put it in the toy pile
elf31 wrapped a elf7combination for child320  wrapped aand threw it in the sleigh 
press for child319 and threw it in the sleigh
elf88 made a quotation for child308 and put it in the toy pile
elf55 made a fly for child307 and put it in the toy pile
elf47 wrapped a laparoscope for child318 and threw it in the sleigh
elf51elf42 wrapped a  wrapped achapel for child317  rule for child316and threw it in the sleigh 
and threw it in the sleigh
elf96elf12  made a drummer for child306wrapped a  and put it in the toy pilegod for child315
 and threw it in the sleigh
elf36 wrapped a youth for child314 and threw it in the sleigh
elf39 elf92wrapped a  wrapped afigure for child313  yard for child312and threw it in the sleigh 
and threw it in the sleigh
elf72 made a nexus for child305 and put it in the toy pile
elf64 made a blackboard for child303elf20  and put it in the toy

elf14 made a chap for child224 and put it in the toy pile
elf82elf19  made amade aelf72   knife-edge for child219hosiery for child220wrapped a   and put it in the toy pileand put it in the toy pilerobe for child240

 and threw it in the sleigh
elf12 made a hypochondria for child218 and put it in the toy pile
elf81 elf78wrapped a  wrapped asection for child238  hovel for child236and threw it in the sleigh 
and threw it in the sleigh
elf56elf21  wrapped awrapped a  furnace for child233canon for child235  and threw it in the sleighand threw it in the sleigh

elf8 made a rock for child217 and put it in the toy pile
elf73 wrapped a hydrofoil for child232 and threw it in the sleigh
elf51 made a jaguar for child216 and put it in the toy pile
elf6 wrapped a original for child231 and threw it in the sleigh
elf61elf2 made a bull for child213 elf37 made a and put it in the toy pile wrapped a
breast for child214 soft for child230  and put it in the toy pileand threw it in the sleigh

elf22 wrapped

elf56 made a month for child127 and put it in the toy pile
elf21 wrapped a nightingale for child138 and threw it in the sleigh
elf54 made a unibody for child126 and put it in the toy pile
elf59elf64  wrapped amade a  scarecrow for child136radiator for child125  and threw it in the sleighand put it in the toy pile

elf6 made a pacemaker for child124 and put it in the toy pile
elf24 wrapped a fairy for child135 and threw it in the sleigh
elf94 made a collar for child123 and put it in the toy pile
elf42 made a kidney for child122 and put it in the toy pile
elf81 made a sombrero for child121 and put it in the toy pile
elf98 wrapped aelf32  background for child133wrapped a  and threw it in the sleighburglar for child132
 and threw it in the sleigh
elf14 wrapped a joy for child131 and threw it in the sleigh
elf73 made a spoon for child120 and put it in the toy pile
elf13 made a bog for child115 and put it in the toy pile
elf84 made a poisoning for child114 and put it in the toy pile
elf1 wra

elf94elf31 made a wrapped a application for child38  and threw it in the sleighpseudoscience for child28
 and put it in the toy pile
elf3 made a gosling for child27 and put it in the toy pile
elf96 wrapped a spelling for child37 and threw it in the sleigh
elf13 made a learning for child25 and put it in the toy pile
elf21 elf12 wrapped amade a  cabbage for child36ketch for child23  and threw it in the sleighand put it in the toy pile

elf52 wrapped a ambition for child35 and threw it in the sleigh
elf82 wrapped aelf7  kettle for child33made a  and threw it in the sleighattorney for child22
 and put it in the toy pile
elf93 made a enjoyment for child20 and put it in the toy pile
elf40 made a serve for child18 and put it in the toy pile
elf18elf98  made awrapped a  promise for child17license for child32  and put it in the toy pileand threw it in the sleigh

elf97 made a accelerator for child16 and put it in the toy pile
elf40elf36 made a  caption for child15wrapped a  and put it in the to